# 🎯 Goal of the Exercise

In this exercise you will learn how to integrate an Azure AI Projects agent with an external Model Context Protocol (MCP) server, run an agent that uses MCP tools, and handle approval and inspection of tool calls.

Objectives:
- Configure and instantiate AIProjectClient and McpTool for MCP server usage.
- Create an agent, start a run that uses MCP tools, and send a user prompt.
- Handle the tool-approval workflow (detect required approvals and submit approvals).
- Inspect run steps, tool calls, and conversation messages.
- Clean up resources by deleting the agent and closing the client.

Prerequisites:
- Set AI_PROJECT_ENDPOINT, MCP_SERVER_URL, MCP_SERVER_LABEL and Azure credentials in ../../.env.
- Install required packages (azure-ai-projects, azure-identity, python-dotenv).

Notebook workflow (high-level):
1. Load env and imports.
2. Initialize AIProjectClient and McpTool.
3. Create agent, thread, and user message; start a run.
4. Poll run status and submit tool approvals when required.
5. List run steps, fetch messages, and perform cleanup.

Expected outcome:
A working agent run demonstrating MCP integration, approval handling, and the ability to inspect tool calls and agent outputs.

### Links to documentation

https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol-samples?pivots=python



### Imports and env:
- Load .env file.
- Import os, time, Azure AI Projects client, DefaultAzureCredential.
- Import models used for agent and MCP tool interactions.

In [1]:
from dotenv import load_dotenv
load_dotenv('../../.env')

# Import necessary libraries

import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import (
    ListSortOrder,
    McpTool,
    RequiredMcpToolCall,
    RunStepActivityDetails,
    SubmitToolApprovalAction,
    ToolApproval,
)

### AI project client & MCP tool setup :
- Read MCP server URL/label from environment variables.
- Instantiate the AIProjectClient (explicit, not using 'with') so cells can run separately.
- Initialize McpTool and add an example allowed tool

In [2]:
# Get MCP server configuration from environment variables and create the client
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://gitmcp.io/Azure/azure-rest-api-specs")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "github")

# Instantiate the client (not using 'with' so cells can be split)
project_client = AIProjectClient(
    endpoint=os.environ["AI_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

# Initialize agent MCP tool
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)

# Add an example allowed tool
search_api_code = "search_azure_rest_api_code"
mcp_tool.allow_tool(search_api_code)
print(f"Allowed tools: {mcp_tool.allowed_tools}")

Allowed tools: ['search_azure_rest_api_code']


### Create agent, thread, message, and start a run :
- Create an agent using the agents client.
- Create a thread and send a user message.
- Update MCP tool headers and create a run using the agent and tool resources.

In [3]:
# Create agent, thread, message and start a run
agents_client = project_client.agents

agent = agents_client.create_agent(
    model="gpt-4o", 
    name="my-mcp-agent",
    instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
    tools=mcp_tool.definitions,
)

print(f"Created agent, ID: {agent.id}")
print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

# Create thread
thread = agents_client.threads.create()
print(f"Created thread, ID: {thread.id}")

#Create message
message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content="Please summarize the Azure REST API specifications Readme",
)
print(f"Created message, ID: {message.id}")

# Prepare headers/approval and create run
mcp_tool.update_headers("SuperSecret", "123456")

# mcp_tool.set_approval_mode("never")  # Uncomment to disable approval requirement
run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
print(f"Created run, ID: {run.id}")

Created agent, ID: asst_ZS6g5WrEUR4IReFzpbexNBBB
MCP Server: github at https://gitmcp.io/Azure/azure-rest-api-specs
Created thread, ID: thread_UtNCJbKmLDnuntczslPkgy1e
Created thread, ID: thread_UtNCJbKmLDnuntczslPkgy1e
Created message, ID: msg_8JPgNR9ZinLTDhriBT7gEmz2
Created message, ID: msg_8JPgNR9ZinLTDhriBT7gEmz2
Created run, ID: run_Na8SHO2mFT5mocZmmfPhSNxC
Created run, ID: run_Na8SHO2mFT5mocZmmfPhSNxC


### Polling and approvals :
- Poll the run status until completion.
- If the run requires action and tool approval, gather tool calls and submit approvals.
- Print the current run status and final result.

In [4]:
# Poll run status and handle tool approvals if required
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(5)
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
        tool_calls = run.required_action.submit_tool_approval.tool_calls
        if not tool_calls:
            print("No tool calls provided - cancelling run")
            agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
            break

        tool_approvals = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredMcpToolCall):
                try:
                    print(f"Approving tool call: {tool_call}")
                    tool_approvals.append(
                        ToolApproval(
                            tool_call_id=tool_call.id,
                            approve=True,
                            headers=mcp_tool.headers,
                        )
                    )
                except Exception as e:
                    print(f"Error approving tool_call {tool_call.id}: {e}")

        print(f"tool_approvals: {tool_approvals}")
        if tool_approvals:
            agents_client.runs.submit_tool_outputs(
                thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
            )

    print(f"Current run status: {run.status}")

print(f"Run completed with status: {run.status}")
if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Approving tool call: {'id': 'call_N6vPrOGlEHwNsEQLV2S2zwf4', 'type': 'mcp', 'arguments': '{"query":"Readme","page":1}', 'name': 'search_azure_rest_api_code', 'server_label': 'github'}
tool_approvals: [{'tool_call_id': 'call_N6vPrOGlEHwNsEQLV2S2zwf4', 'approve': True, 'headers': {'SuperSecret': '123456'}}]
Current run status: RunStatus.REQUIRES_ACTION
Current run status: RunStatus.REQUIRES_ACTION
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.COMPLETED
Run completed with status: RunStatus.COMPLETED
Current run status: RunStatus.COMPLETED
Run completed with status: RunStatus.COMPLETED


### Display results, messages and perform cleanup:
- List run steps and display any tool calls or function parameters.
- Fetch and print conversation messages.
- Finally delete the agent and close the client.

In [5]:
# Display run steps and tool calls

run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  MCP Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

    if isinstance(step_details, RunStepActivityDetails):
        for activity in step_details.activities:
            for function_name, function_definition in activity.tools.items():
                print(
                    f'  The function {function_name} with description "{function_definition.description}" will be called.:'
                )
                if len(function_definition.parameters) > 0:
                    print("  Function parameters:")
                    for argument, func_argument in function_definition.parameters.properties.items():
                        print(f"      {argument}")
                        print(f"      Type: {func_argument.type}")
                        print(f"      Description: {func_argument.description}")
                else:
                    print("This function has no parameters")
    print()

# Fetch and log all messages
messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
print("\nConversation:")
print("-" * 50)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role.upper()}: {last_text.text.value}")
        print("-" * 50)


# Clean-up and delete the agent once the run is finished.
# NOTE: Comment out this line if you plan to reuse the agent later.
agents_client.delete_agent(agent.id)
print("Deleted agent")

# Close the client explicitly since we removed the 'with' context
try:
    project_client.close()
except Exception:
    pass

Step step_15W8Z6Q5t2tVJF4LdrFycTBo status: completed

Step step_8oXZ0QjABs673yke8WuypYCP status: completed
  MCP Tool calls:
    Tool Call ID: call_N6vPrOGlEHwNsEQLV2S2zwf4
    Type: mcp


Conversation:
--------------------------------------------------

Conversation:
--------------------------------------------------
USER: Please summarize the Azure REST API specifications Readme
--------------------------------------------------
ASSISTANT: The Azure REST API specifications repository contains various README files specific to different tools, APIs, and resources. These README files typically provide guidance on:

1. **Linting and Verification**: Guidelines and scripts like lint-diff and `Verify-Readme.ps1` to validate API spec formats or documentation.

2. **Resource-Specific APIs**: Each resource or service (e.g., Azure Orbital, DNS, Databricks, KeyVault, Digital Twins, etc.) includes separate README files outlining usage, details for API endpoints, and implementation instructions.

